### Cell 1 (Imports): 
프로젝트 전역에서 사용하는 라이브러리를 단 한 번만 정의하여 의존성 충돌을 방지합니다.

In [1]:
!pip -q install "openai>=1.40.0" python-dotenv pandas openpyxl numpy

In [2]:
# ===== Cell 1: Imports (ONLY HERE) =====
import os, re, json
from datetime import datetime, timezone
from typing import Optional, Dict, List, Tuple, Any

import numpy as np
import pandas as pd
from dotenv import load_dotenv, find_dotenv

### Cell 2 (Constants): 
시스템의 '뇌' 역할을 하는 정책들입니다. UNIT_TOKENS(단위 사전)와 PATTERNS_BY_SLOT(컬럼명 패턴)을 이곳에서 관리하여,<br>
비즈니스 로직 수정 없이 상수 값 변경만으로 시스템 운영이 가능하게 설계되었습니다.

In [3]:
# ===== Cell 2: Constants / Policies / Logging / Env =====

DEBUG = True

def log(layer: str, msg: str, **kwargs):
    if not DEBUG:
        return
    print(f"[{layer}] {msg}")
    if kwargs:
        for k, v in kwargs.items():
            print(f"  - {k}: {v}")

def now_utc_iso() -> str:
    return datetime.now(timezone.utc).replace(microsecond=0).isoformat().replace("+00:00", "Z")

# .env 로딩 (필요 시)
load_dotenv(find_dotenv(), override=False)
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY가 .env에 없습니다."
log("INIT", ".env loaded", OPENAI_API_KEY="SET")

# 지원 슬롯
SLOT_NAMES = ["electricity_usage", "citygas_usage", "water_usage"]

# 단위 토큰 표준화(키: 감지 토큰, 값: 표준 단위 문자열)
UNIT_TOKENS = {
    "kwh": "kWh", "mwh": "MWh", "wh": "Wh",
    "mj": "MJ", "gj": "GJ",
    "m3": "m3", "㎥": "m3",
    "tco2e": "tCO2e", "co2e": "tCO2e",
    "kvarh": "kVarh",
}

# slotName별 컬럼 패턴
PATTERNS_BY_SLOT = {
    "electricity_usage": {
        "date":  [r"\btimestamp\b", r"\bdatetime\b", r"\bdate\b", r"\btime\b", "일시", "시각", "연월일", "년월일", "일자", "검침일"],
        "flow":  ["사용량", "전력", "전기사용", "usage", r"\bconsumption\b", r"\bpower\b"],
        "cum":   ["누적", "cumulative", "meter", "계량"],
        "unit":  [r"\bunit\b", r"\buom\b", "단위", "kwh", "mwh", "wh", "mj", "gj", "kvarh"],
    },
    "citygas_usage": {
        "date":  [r"\btimestamp\b", r"\bdatetime\b", r"\bdate\b", r"\btime\b", "일시", "시각", "연월일", "년월일", "일자", "검침일"],
        "flow":  ["사용량", "가스", "도시가스", r"\bflow\b", r"\bvolume\b", "usage"],
        "cum":   ["누적", "cumulative", "meter", "계량"],
        "unit":  [r"\bunit\b", r"\buom\b", "단위", "m3", "㎥", "gj", "mj", "tco2e", "co2e"],
    },
    "water_usage": {
        "date":  [r"\btimestamp\b", r"\bdatetime\b", r"\bdate\b", r"\btime\b", "일시", "시각", "연월일", "년월일", "일자", "검침일"],
        "flow":  ["사용량", "수도", "용수", r"\bflow\b", r"\bvolume\b", "usage"],
        "cum":   ["누적", "cumulative", "meter", "계량"],
        "unit":  [r"\bunit\b", r"\buom\b", "단위", "m3", "㎥"],
    },
}

# slot별 예상 단위
EXPECTED_UNITS_BY_SLOT = {
    "electricity_usage": {"kWh"},
    "citygas_usage": {"m3"},
    "water_usage": {"m3"},
}

# 단위 토큰이 전혀 없을 때 fallback
SLOT_DEFAULT_UNIT = {
    "electricity_usage": "kWh",
    "citygas_usage": "m3",
    "water_usage": "m3",
}

# 최종 표준 컬럼명 규칙
SLOT_OUTPUT_SCHEMA = {
    "electricity_usage": {
        "timestamp": "timestamp",
        "flow": "flow_kwh",
        "cum": None,  # 전기는 기본적으로 cum 없음
    },
    "citygas_usage": {
        "timestamp": "timestamp",
        "flow": "flow_m3",
        "cum": None,  # 가스는 cum 필수 아님(필요 시 확장 가능)
    },
    "water_usage": {
        "timestamp": "timestamp",
        "flow": "flow_m3",
        "cum": "cumulative_meter_m3",  # 수도는 cum이 있으면 core로 처리 가능
    },
}

# 파싱률 임계치
TS_THRESHOLD = 0.95
NUM_THRESHOLD = 0.95

# 정책
FAIL_IF_UNIT_UNRESOLVED = False
FAIL_IF_DUP_TS = True

# LLM 옵션
#USE_LLM = False  # 운영에서는 기본 False, 필요 시 True
USE_LLM = True

# unit_schema label
TIME_UNIT_LABEL = "time"
UNKNOWN_UNIT_LABEL = "-"


[INIT] .env loaded
  - OPENAI_API_KEY: SET


### Cell 3 (Utilities): 
정규표현식 점수 계산(rule_score), 날짜 정규화(normalize_datetime_series) 등 재사용성이 높은 '순수 함수'들의 집합입니다. 
<br>특히 build_output_schema는 최종 데이터 규격을 통일하는 아주 중요한 역할을 합니다.

In [4]:
# ===== Cell 3: Utilities (NO imports, NO constant re-definitions) =====

def rule_score(colname: str, patterns: List[str]) -> int:
    s = str(colname).lower()
    score = 0
    for p in patterns:
        if p.startswith(r"\b") or "(" in p or "[" in p:
            if re.search(p, s, flags=re.IGNORECASE):
                score += 1
        else:
            if p.lower() in s:
                score += 1
    return score

def parse_rate_numeric(series: pd.Series) -> float:
    cleaned = series.astype(str).str.replace(",", "", regex=False).str.strip()
    parsed = pd.to_numeric(cleaned, errors="coerce")
    return float(parsed.notna().mean()) if len(parsed) else 0.0

def detect_unit_from_name(colname: str) -> Optional[str]:
    s = str(colname).lower()
    for k, u in UNIT_TOKENS.items():
        if k in s:
            return u
    return None

def detect_unit_from_unit_column(unit_series: pd.Series) -> Optional[str]:
    values = unit_series.astype(str).str.lower().dropna().tolist()
    hits = []
    for v in values[:500]:
        for k, u in UNIT_TOKENS.items():
            if k in v:
                hits.append(u)
    if not hits:
        return None
    return pd.Series(hits).mode().iloc[0]

def parse_period(period_start: str, period_end: str) -> Tuple[pd.Timestamp, pd.Timestamp]:
    ps = pd.to_datetime(period_start, errors="raise").tz_localize(None)
    pe = pd.to_datetime(period_end, errors="raise").tz_localize(None)
    if pe < ps:
        raise ValueError("period_end가 period_start보다 빠릅니다.")
    return ps, pe

def normalize_datetime_series(raw: pd.Series, period_start: pd.Timestamp) -> Tuple[pd.Series, float, Dict[str, Any], pd.Series]:
    s = raw.astype(str).str.strip()

    ts1 = pd.to_datetime(s, errors="coerce")
    if ts1.dt.tz is not None:
        ts1 = ts1.dt.tz_localize(None)

    def _fix_one(x: str) -> Optional[pd.Timestamp]:
        if x is None: 
            return None
        x = str(x).strip()
        if not x:
            return None

        x2 = (x.replace("년", "/").replace("월", "/").replace("일", " ")
               .replace("시", ":").replace("분", ""))
        x2 = x2.replace(".", "/").replace("-", "/").replace(",", " ")
        x2 = re.sub(r"\s+", " ", x2).strip()

        nums = re.findall(r"\d+", x2)
        if len(nums) < 2:
            return None

        if len(nums[0]) == 4:
            year = int(nums[0]); idx = 1
        else:
            year = int(period_start.year); idx = 0

        # 간단한 케이스 보정
        try:
            month = int(nums[idx]); day = int(nums[idx + 1])
            hour = int(nums[idx + 2]) if len(nums) >= idx + 3 else 0
            minute = int(nums[idx + 3]) if len(nums) >= idx + 4 else 0
            if 0 <= hour < 24 and 0 <= minute < 60:
                return pd.Timestamp(year=year, month=month, day=day, hour=hour, minute=minute)
        except:
            return None
        return None

    ts2 = ts1.copy()
    mask_na = ts2.isna()
    filled = 0
    if mask_na.any():
        fixed = s[mask_na].map(_fix_one)
        ts2.loc[mask_na] = pd.to_datetime(fixed, errors="coerce")
        filled = int(mask_na.sum())

    rate = float(ts2.notna().mean()) if len(ts2) else 0.0
    normalized_str = ts2.dt.strftime("%Y/%m/%d %H:%M")
    debug = {"parse_rate": rate, "filled_from_custom": filled}
    return ts2, rate, debug, normalized_str

def detect_time_granularity(ts_parsed: pd.Series) -> Optional[str]:
    ts = ts_parsed.dropna().sort_values()
    if len(ts) < 3:
        return None
    deltas = ts.diff().dropna().dt.total_seconds()
    if deltas.empty:
        return None
    mode_sec = float(deltas.mode().iloc[0])

    if abs(mode_sec - 600) <= 60: return "10min"
    if abs(mode_sec - 900) <= 60: return "15min"
    if abs(mode_sec - 1800) <= 120: return "30min"
    if abs(mode_sec - 3600) <= 120: return "hourly"
    if abs(mode_sec - 86400) <= 3600: return "day"
    if abs(mode_sec - 604800) <= 86400: return "week"
    return None

def granularity_to_pandas_freq(gran: str) -> Optional[str]:
    mapping = {"10min": "10min", "15min": "15min", "30min": "30min", "hourly": "h", "day": "d"}
    return mapping.get(gran)

#  validated_fields / unit_schema 생성은 아래 하나로 통일합니다.
def build_output_schema(
    df: pd.DataFrame,
    slotName: str,
    date_col: str,
    flow_col: str,
    cum_col: Optional[str],
    resolved_flow_unit: str
) -> Tuple[List[str], List[str], Dict[str, str]]:
    """
    - core 컬럼: timestamp, flow_*, (cum이 선택되면 cum도 core)
    - non-core 컬럼: unit_schema는 무조건 "-"
    - validated_fields: 최종 출력 컬럼명 리스트
    - rename_map: 원본컬럼 -> 최종컬럼명
    """
    schema = SLOT_OUTPUT_SCHEMA[slotName]
    rename_map: Dict[str, str] = {}

    out_ts = "timestamp"
    out_flow = schema["flow"]
    out_cum = schema.get("cum", None)  # water는 cumulative_meter_m3 가능

    # core rename
    rename_map[date_col] = out_ts
    rename_map[flow_col] = out_flow

    cum_is_core = False
    if cum_col is not None and out_cum is not None:
        rename_map[cum_col] = out_cum
        cum_is_core = True

    # validated_fields = [core...] + [others...]
    validated_fields: List[str] = [out_ts, out_flow]
    if cum_is_core:
        validated_fields.append(out_cum)

    # others keep original names (단, core와 이름 충돌하면 __orig)
    reserved = set(validated_fields)
    for c in df.columns:
        c = str(c)
        if c in rename_map:
            continue
        out_name = c
        if out_name in reserved:
            out_name = f"{out_name}__orig"
        rename_map[c] = out_name
        reserved.add(out_name)
        validated_fields.append(out_name)

    # unit_schema 규칙: core만 단위, 나머지는 "-"
    unit_schema: List[str] = []
    for name in validated_fields:
        if name == out_ts:
            unit_schema.append(TIME_UNIT_LABEL)
        elif name == out_flow:
            unit_schema.append(resolved_flow_unit)
        elif cum_is_core and name == out_cum:
            unit_schema.append("m3")
        else:
            unit_schema.append(UNKNOWN_UNIT_LABEL)

    return validated_fields, unit_schema, rename_map


### Cell 4 (Output Helpers): 
아웃풋 규격(JSON)을 강제합니다. 성공(PASS)과 실패(FAIL)의 구조를 엄격(Strict)하게 유지하여, 프론트엔드나 API 연동 시 에러 발생 가능성을 0으로 만듭니다.

In [5]:
# ===== Cell 4: Strict Output Helpers =====

def pass_output_strict(file_path: str, time_granularity: str, unit_schema: list, validated_fields: list):
    return {
        "status": "PASS",
        "file_path": file_path,
        "payload": {
            "time_granularity": time_granularity,
            "unit_schema": unit_schema,
            "validated_fields": validated_fields
        },
        "processed_at": now_utc_iso()
    }

def fail_output_strict(file_path: str, code: str, message: str, location: str):
    return {
        "status": "FAIL",
        "error": {"code": code, "message": message, "location": location},
        "file_path": file_path,
        "processed_at": now_utc_iso()
    }

def print_full(result: dict):
    print(json.dumps(result, ensure_ascii=False, indent=2))


### Cell 5 (L0): 
입력 데이터의 '형식'을 검사합니다. 가장 비용이 적게 드는 검증으로, 잘못된 요청을 입구에서 컷트(Cut)합니다.

In [6]:
# ===== Cell 5: L0 input sanity =====

def layer0_input_sanity(obj: dict) -> Tuple[Optional[dict], Optional[dict]]:
    layer = "L0"

    if not isinstance(obj, dict):
        log(layer, "BAD_INPUT_TYPE", got=str(type(obj)))
        return None, {"code":"BAD_INPUT", "message":"입력이 dict가 아닙니다.", "location":"input"}

    required = ["slotName", "kind", "ext", "period_start", "period_end", "dataframe"]
    missing = [k for k in required if k not in obj]
    if missing:
        log(layer, "MISSING_KEYS", missing=missing)
        return None, {"code":"MISSING_KEYS", "message":f"필수 키 누락: {missing}", "location":"input"}

    slot = obj["slotName"]
    if slot not in SLOT_NAMES:
        log(layer, "UNKNOWN_SLOT", slotName=slot)
        return None, {"code":"UNKNOWN_SLOT", "message":f"지원하지 않는 slotName: {slot}", "location":"input.slotName"}

    df = obj["dataframe"]
    if not isinstance(df, pd.DataFrame):
        log(layer, "BAD_DF_TYPE", got=str(type(df)))
        return None, {"code":"BAD_DF", "message":"dataframe이 pandas.DataFrame이 아닙니다.", "location":"input.dataframe"}

    if df.shape[0] < 1 or df.shape[1] < 2:
        log(layer, "NO_DATA", shape=df.shape)
        return None, {"code":"NO_DATA", "message":f"데이터 행/열이 부족합니다. shape={df.shape}", "location":"input.dataframe"}

    try:
        ps, pe = parse_period(obj["period_start"], obj["period_end"])
    except Exception as e:
        log(layer, "BAD_PERIOD", exc=type(e).__name__)
        return None, {"code":"BAD_PERIOD", "message":f"기간 파싱 실패: {type(e).__name__}", "location":"input.period_start/end"}

    log(layer, "PASS", slotName=slot, shape=df.shape, period_start=str(ps), period_end=str(pe))

    normalized = dict(obj)
    normalized["_period_start_ts"] = ps
    normalized["_period_end_ts"] = pe
    return normalized, None


### Cell 6 (L1): 
지저분한 엑셀 데이터를 정제합니다. 특히 strip() 처리 후 중복 컬럼이 발생하는 예외 상황을 잡아내어 데이터 오염을 방지합니다.

In [7]:
# ===== Cell 6: L1 normalize DF (수정본) =====

def layer1_normalize_df(df: pd.DataFrame) -> Tuple[Optional[pd.DataFrame], Optional[dict]]:
    layer = "L1"
    df2 = df.copy()
    
    # 컬럼명의 앞뒤 공백 제거
    cols = [str(c).strip() for c in df2.columns]

    # 1. 빈 컬럼명 검사
    if any(c == "" for c in cols):
        log(layer, "EMPTY_COLUMN_NAME", columns=cols)
        return None, {"code":"L1_INVALID_COLUMNS", "message":"빈 컬럼명이 존재합니다(strip 후 '').", "location":"df.columns"}

    # 2. 중복 컬럼명 검사 (duplicated 함수 추가됨)
    if pd.Series(cols).duplicated().any(): 
        log(layer, "DUPLICATE_COLUMNS_AFTER_NORMALIZE", columns=cols)
        return None, {"code":"L1_INVALID_COLUMNS", "message":"L1 정규화 이후 중복 컬럼명이 발생했습니다(strip 후 동일).", "location":"df.columns"}

    df2.columns = cols
    log(layer, "PASS", columns=cols)
    return df2, None

### Cell 7 (LLM Classify): 
룰 기반으로 찾기 어려운 애매한 컬럼명을 LLM을 통해 분류하는 인터페이스입니다. 현재는 안전하게 비워져(Empty) 있어 시스템 안정성을 높였습니다.

In [8]:
# ===== Cell 7: LLM classify (필요시 작동하는 실제 로직) =====
from openai import OpenAI

def llm_classify_columns_min_tokens(columns: List[str], df: pd.DataFrame, slotName: str) -> dict:
    # USE_LLM이 False면 바로 빈 결과 반환 (이중 방어)
    if not USE_LLM:
        return {"mapping": {}}
    
    # 1. API 클라이언트 초기화 (.env의 키 사용)
    client = OpenAI() 
    
    # 2. 토큰 절약 및 보안을 위해 데이터 상단 3줄만 텍스트로 변환
    sample_data = df.head(3).to_csv(index=False)
    
    # 3. 프롬프트 구성: 설명은 배제하고 JSON 결과만 요구 (Strict Output)
    prompt = f"""
    Analyze the following CSV columns for a sustainability report ({slotName}).
    Columns: {columns}
    Sample Data:
    {sample_data}
    
    Identify which column represents:
    - 'date': Timestamp or date
    - 'flow': Main usage/consumption value
    - 'cum': Cumulative meter value (if exists)
    - 'unit': Unit of measurement (if exists)
    
    "If a column seems to be noise or irrelevant data, map it as 'other' and do not assign it to 'flow' or 'cum'."
    
    Return ONLY a JSON object: {{"mapping": {{"column_name": "label"}}}}
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", # 저렴하고 빠른 모델 권장
            messages=[{"role": "user", "content": prompt}],
            response_format={ "type": "json_object" }
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        log("LLM_ERROR", str(e))
        return {"mapping": {}}

### Cell 8 (L2: Find Columns): 
'점수제 -> LLM(선택) -> 값 샘플링(Fallback)'의 3단계 방어 로직을 통해, 어떤 회사 파일이 들어와도 날짜와 사용량 컬럼을 정확히 찾아냅니다.

In [9]:
# ===== Cell 8: L2 find columns =====

def layer2_find_columns(
    df: pd.DataFrame,
    slotName: str,
    topn: int = 10,
    auto_llm: bool = True,
    force_llm: bool = False
) -> Tuple[Optional[str], Dict[str, Optional[str]], List[str], Optional[dict], dict]:

    layer = "L2"
    pats = PATTERNS_BY_SLOT[slotName]
    cols = [str(c) for c in df.columns]

    scored = []
    for c in cols:
        scored.append((
            c,
            rule_score(c, pats["date"]),
            rule_score(c, pats["flow"]),
            rule_score(c, pats["unit"]),
            rule_score(c, pats["cum"])
        ))

    date_candidates = [c for c, sd, _, _, _ in sorted(scored, key=lambda x: x[1], reverse=True)[:topn] if sd > 0]
    flow_candidates = [c for c, _, sf, _, _ in sorted(scored, key=lambda x: x[2], reverse=True)[:topn] if sf > 0]
    cum_candidates  = [c for c, _, _, _, sc in sorted(scored, key=lambda x: x[4], reverse=True)[:topn] if sc > 0]
    unit_cols       = [c for c, _, _, su, _ in scored if su > 0]

    debug = {
        "scored_top10": sorted(scored, key=lambda x: sum(x[1:]), reverse=True)[:10],
        "date_candidates": date_candidates,
        "flow_candidates": flow_candidates,
        "cum_candidates": cum_candidates,
        "unit_cols": unit_cols,
        "used_llm": False,
        "llm_reason": None,
        "llm_raw": None
    }

    date_col = date_candidates[0] if date_candidates else None
    flow_col = flow_candidates[0] if flow_candidates else None
    cum_col  = cum_candidates[0] if cum_candidates else None

    # (1) LLM 트리거 판단
    need_llm = False
    if force_llm:
        need_llm = True
        debug["llm_reason"] = "force_llm"
        log(layer, "AUTO_LLM_TRIGGERED", reason="force_llm", USE_LLM=USE_LLM)
    elif auto_llm and (not date_col or not flow_col):
        # 룰이 실패했고, auto_llm 켜져 있으면 LLM을 “시도”할 수 있음
        need_llm = True
        debug["llm_reason"] = "rule_missing"
        log(layer, "AUTO_LLM_TRIGGERED", reason="rule_missing", USE_LLM=USE_LLM)

    # (2) LLM 실행(옵션)
    if need_llm and USE_LLM:
        debug["used_llm"] = True
        llm_res = llm_classify_columns_min_tokens(cols, df, slotName)
        debug["llm_raw"] = llm_res

        mapping = (llm_res.get("mapping", {}) or {})
        date_llm = [c for c, lab in mapping.items() if lab == "date" and c in cols]
        flow_llm = [c for c, lab in mapping.items() if lab == "flow" and c in cols]
        cum_llm  = [c for c, lab in mapping.items() if lab == "cum" and c in cols]
        unit_llm = [c for c, lab in mapping.items() if lab == "unit" and c in cols]

        date_col = date_col or (date_llm[0] if date_llm else None)
        flow_col = flow_col or (flow_llm[0] if flow_llm else None)
        cum_col  = cum_col  or (cum_llm[0] if cum_llm else None)
        unit_cols = list(set(unit_cols + unit_llm))

    # (3) fallback: 값 형태로 찾기
    if not date_col:
        date_rates = sorted(
            [(c, float(pd.to_datetime(df[c], errors="coerce").notna().mean())) for c in cols],
            key=lambda x: x[1], reverse=True
        )
        debug["date_fallback_top5"] = date_rates[:5]
        date_col = date_rates[0][0] if (date_rates and date_rates[0][1] >= TS_THRESHOLD) else None

    if not flow_col:
        val_rates = sorted([(c, parse_rate_numeric(df[c])) for c in cols], key=lambda x: x[1], reverse=True)
        debug["flow_fallback_top5"] = val_rates[:5]
        flow_col = val_rates[0][0] if (val_rates and val_rates[0][1] >= NUM_THRESHOLD) else None

    if not date_col or not flow_col:
        return None, {"flow": None, "cum": None}, unit_cols, {
            "code":"REQUIRED_FIELD_NOT_FOUND",
            "message":"필수 컬럼(date/flow)을 찾지 못했습니다.",
            "location":"columns"
        }, debug

    log(layer, "picked", date_col=date_col, flow_col=flow_col, cum_col=cum_col, unit_cols=unit_cols)
    return date_col, {"flow": flow_col, "cum": cum_col}, unit_cols, None, debug


### Cell 9 (L3: Value Validation): 
데이터 값이 진짜 숫자인지, 날짜인지 정밀 검사합니다. 특히 resolved_unit을 통해 잘못된 단위(ex. 전기인데 m3)가 들어오는 것을 차단합니다.

In [10]:
# ===== Cell 9: L3 validate values =====

def layer3_validate_values(
    df: pd.DataFrame,
    slotName: str,
    date_col: str,
    value_cols: Dict[str, Optional[str]],
    unit_cols: List[str],
    period_start: pd.Timestamp
) -> Tuple[Optional[dict], dict]:

    layer = "L3"
    debug: Dict[str, Any] = {}

    flow_col = value_cols["flow"]

    ts_parsed, ts_rate, ts_dbg, ts_norm_str = normalize_datetime_series(df[date_col], period_start=period_start)
    flow_num_rate = parse_rate_numeric(df[flow_col])

    time_gran = detect_time_granularity(ts_parsed)

    debug["ts_rate"] = ts_rate
    debug["flow_num_rate"] = flow_num_rate
    debug["time_granularity"] = time_gran

    if ts_rate < TS_THRESHOLD:
        return {"code":"TS_PARSE_LOW", "message":f"날짜 파싱률 낮음(ts_rate={ts_rate:.2f})", "location":f"col:{date_col}"}, debug
    if flow_num_rate < NUM_THRESHOLD:
        return {"code":"VALUE_PARSE_LOW", "message":f"숫자 파싱률 낮음(num_rate={flow_num_rate:.2f})", "location":f"col:{flow_col}"}, debug
    if time_gran is None:
        return {"code":"TIME_GRAIN_UNKNOWN", "message":"시간 해상도 판별 불가", "location":f"col:{date_col}"}, debug

    # 단위 결정
    resolved_unit = None

    # 1) unit 컬럼이 있으면 그 값에서 탐지
    for uc in unit_cols or []:
        u = detect_unit_from_unit_column(df[uc])
        if u:
            resolved_unit = u
            break

    # 2) flow 컬럼명에서 탐지 (Usage_kWh 같은 케이스)
    if resolved_unit is None:
        u = detect_unit_from_name(flow_col)
        if u:
            resolved_unit = u

    # 3) slot 기본 단위 fallback
    if resolved_unit is None:
        if FAIL_IF_UNIT_UNRESOLVED:
            return {"code":"UNIT_UNRESOLVED", "message":"단위 판별 불가", "location":f"col:{flow_col}"}, debug
        resolved_unit = SLOT_DEFAULT_UNIT[slotName]

    expected_units = EXPECTED_UNITS_BY_SLOT[slotName]
    if resolved_unit not in expected_units:
        return {
            "code":"UNIT_MISMATCH",
            "message":f"단위 불일치: got={resolved_unit}, expected={list(expected_units)[0]}",
            "location":f"col:{flow_col}"
        }, debug

    debug["_ts_parsed"] = ts_parsed
    debug["resolved_unit"] = resolved_unit
    return None, debug


### Cell 10 (L4: Period Coverage): 
가장 까다로운 단계입니다. 1시간 간격 데이터인데 12시 데이터가 없다면 바로 잡아냅니다. 환경 공시 보고서의 신뢰도를 결정짓는 핵심 셀입니다.

In [11]:
# ===== Cell 10: L4 period coverage =====

def granularity_to_pandas_freq(gran: str) -> Optional[str]:
    mapping = {"10min": "10min", "15min": "15min", "30min": "30min", "hourly": "h", "day": "d"}
    return mapping.get(gran)

def align_ts_for_compare(ts: pd.Series, gran: str) -> pd.Series:
    if gran in ("10min", "15min", "30min"):
        mins = int(gran.replace("min", ""))
        return ts.dt.floor(f"{mins}min")
    if gran == "hourly":
        return ts.dt.floor("h")
    if gran == "day":
        return ts.dt.floor("d")
    return ts

def layer4_validate_period_coverage(
    df: pd.DataFrame,
    ts_parsed: pd.Series,
    flow_col: str,
    period_start: pd.Timestamp,
    period_end: pd.Timestamp,
    granularity: str
) -> Tuple[Optional[dict], dict]:

    layer = "L4"
    debug: Dict[str, Any] = {}

    ts_aligned = align_ts_for_compare(ts_parsed, granularity)

    mask = (ts_aligned >= period_start) & (ts_aligned <= period_end)
    df_in = df.loc[mask].copy()
    ts_in = ts_aligned.loc[mask]

    debug["rows_in_period"] = int(df_in.shape[0])
    if df_in.shape[0] == 0:
        return {"code":"PERIOD_OUT_OF_RANGE", "message":"지정 기간 내 데이터가 없습니다.", "location":"period"}, debug

    dup_count = int(pd.Series(ts_in.dropna().values).duplicated().sum())
    debug["duplicate_ts_count"] = dup_count
    if FAIL_IF_DUP_TS and dup_count > 0:
        return {"code":"DUPLICATE_TIMESTAMPS", "message":"기간 내 중복 timestamp 존재", "location":"timestamp"}, debug

    freq = granularity_to_pandas_freq(granularity)
    if freq is None:
        return {"code":"TIME_GRAIN_UNKNOWN", "message":"시간 해상도 판별 불가(L4)", "location":"time_granularity"}, debug

    expected = pd.date_range(start=period_start, end=period_end, freq=freq)
    exp_set = set(expected.tolist())
    act_set = set(ts_in.dropna().tolist())

    missing = sorted(list(exp_set - act_set))
    if missing:
        return {"code":"PERIOD_MISSING_TIMESTAMPS", "message":"기간 내 timestamp 누락", "location":"timestamp"}, debug

    val_num = pd.to_numeric(df_in[flow_col].astype(str).str.replace(",", "", regex=False).str.strip(), errors="coerce")
    if int(val_num.isna().sum()) > 0:
        return {"code":"PERIOD_VALUE_MISSING", "message":"기간 내 flow 값 누락", "location":f"col:{flow_col}"}, debug

    log(layer, "PASS", rows_in_period=debug["rows_in_period"])
    return None, debug


### [Cell 11] 메인 오케스트레이터 (validate_structured_upstream)
L0부터 L4까지의 모든 셀을 순서대로 실행하며 결과를 조합합니다. 각 레이어의 리턴값을 체크하여 중간에 문제가 생기면 즉시 중단(Abort)하고 에러를 리포팅하는 지휘자 역할을 합니다.

In [12]:
# ===== Cell 11 validate_structured_upstream =====


def validate_structured_upstream(obj: dict, file_path: str = "upstream.xlsx"):
    log("MAIN", "start", slotName=obj.get("slotName"), file_path=file_path)

    # L0
    norm, err0 = layer0_input_sanity(obj)
    if err0:
        return fail_output_strict(file_path, err0["code"], err0["message"], err0["location"])

    slotName = norm["slotName"]
    df_raw = norm["dataframe"]
    ps, pe = norm["_period_start_ts"], norm["_period_end_ts"]

    # L1
    df, err1 = layer1_normalize_df(df_raw)
    if err1:
        return fail_output_strict(file_path, err1["code"], err1["message"], err1["location"])

    # L2
    date_col, value_cols, unit_cols, err2, _ = layer2_find_columns(df, slotName=slotName)
    if err2:
        return fail_output_strict(file_path, err2["code"], err2["message"], err2["location"])

    # L3
    err3, dbg3 = layer3_validate_values(df, slotName, date_col, value_cols, unit_cols, ps)
    if err3:
        return fail_output_strict(file_path, err3["code"], err3["message"], err3["location"])

    # L4
    err4, _ = layer4_validate_period_coverage(df, dbg3["_ts_parsed"], value_cols["flow"], ps, pe, dbg3["time_granularity"])
    if err4:
        return fail_output_strict(file_path, err4["code"], err4["message"], err4["location"])

    # 여기서 output 스키마/단위 스키마를 "하나의 함수"로 통일 생성
    validated_fields, unit_schema, rename_map = build_output_schema(
        df=df,
        slotName=slotName,
        date_col=date_col,
        flow_col=value_cols["flow"],
        cum_col=value_cols.get("cum"),
        resolved_flow_unit=dbg3["resolved_unit"]
    )

    payload_time_gran = dbg3["time_granularity"]
    return pass_output_strict(file_path, payload_time_gran, unit_schema, validated_fields)


### 테스트 셀들(12~19)
Cell 12 — 테스트 데이터 생성기(현실성 + 규모) <br>
역할: 
전기(15분, 8832행), 가스/수도(시간, 2208행) 같은 현실적 규모로 df를 만든다.<br>
“파일 I/O 없이도” 파이프라인 검증 가능.

In [13]:
# ===== Cell 12: Test data generators =====

def make_ts_range(period_start: str, period_end: str, freq: str) -> pd.DatetimeIndex:
    ps = pd.to_datetime(period_start)
    pe = pd.to_datetime(period_end)
    return pd.date_range(ps, pe, freq=freq)

def gen_electricity_df(period_start: str, period_end: str) -> pd.DataFrame:
    ts = make_ts_range(period_start, period_end, "15min")
    n = len(ts)
    df = pd.DataFrame({
        "date": ts,
        "Usage_kWh": np.round(np.random.uniform(50, 180, n), 3),
        "Lagging_Current_Reactive.Power_kVarh": np.round(np.random.uniform(0, 30, n), 3),
        "Leading_Current_Reactive_Power_kVarh": np.round(np.random.uniform(0, 30, n), 3),
        "Lagging_Current_Power_Factor": np.round(np.random.uniform(0.7, 1.0, n), 3),
        "Leading_Current_Power_Factor": np.round(np.random.uniform(0.7, 1.0, n), 3),
    })
    return df

def gen_water_df(period_start: str, period_end: str) -> pd.DataFrame:
    ts = make_ts_range(period_start, period_end, "h")
    n = len(ts)
    flow = np.round(np.random.uniform(5, 40, n), 3)
    cum = np.round(np.cumsum(flow), 3)
    df = pd.DataFrame({
        "timestamp": ts,
        "flow_m3": flow,
        "cumulative_meter_m3": cum
    })
    return df

def gen_citygas_df(period_start: str, period_end: str) -> pd.DataFrame:
    ts = make_ts_range(period_start, period_end, "h")
    n = len(ts)
    flow = np.round(np.random.uniform(10, 60, n), 3)
    calorific = np.round(np.random.uniform(38, 43, n), 3)  # MJ/m3 근사
    energy = np.round(flow * calorific, 3)                  # MJ
    co2e = np.round(energy * 0.000056, 6)                   # 임의 계수(테스트용)
    df = pd.DataFrame({
        "timestamp": ts,
        "flow_m3": flow,
        "calorific_MJ_per_m3": calorific,
        "energy_MJ": energy,
        "CO2e_t": co2e,
    })
    return df


Cell 13 — PASS 테스트 3종(전기/수도/가스)
역할

“실제 운영 성공 케이스” 3개를 한 번에 보여줌

In [14]:
# ===== Cell 13: Real File Validation (Path Adjusted) =====

# 1. 경로 설정 (노트북 파일 위치인 notebooks/ 에서 상위로 이동 후 data/samples/E/ 로 접근)
BASE_DATA_PATH = "../data/samples/E"

test_configs = [
    {
        "slot": "electricity_usage",
        "file_name": "성광벤드_전기요금_2025년_4분기.xlsx",
        "label": "전기요금"
    },
    {
        "slot": "water_usage",
        "file_name": "성광벤드_수도요금_2025년_4분기.xlsx",
        "label": "수도요금"
    },
    {
        "slot": "citygas_usage",
        "file_name": "성광벤드_도시가스요금_2025년_4분기.xlsx",
        "label": "도시가스요금"
    }
]

# 2025년 4분기 표준 기간 설정 (사용자 요청 기간 반영)
# 데이터의 마지막 시각이 00:00일 경우와 23:59일 경우를 대비해 설정 확인 필요
PERIOD_START = "2025-10-01T00:00:00"
PERIOD_END   = "2025-12-31T23:59:59"

print(f"[REAL-FILE TEST] 검증 시작: {now_utc_iso()}")
print("-" * 60)

for config in test_configs:
    full_path = os.path.join(BASE_DATA_PATH, config["file_name"])
    label = config["label"]
    
    print(f"\n[PROCESSING] {label} ({config['file_name']}) 검사 중...")
    
    # [Step 1] 파일 존재 여부 확인
    if not os.path.exists(full_path):
        print(f"  에러: 파일을 찾을 수 없습니다. 경로: {full_path}")
        continue
        
    try:
        # [Step 2] pandas 데이터 로드
        raw_df = pd.read_excel(full_path)
        
        # [Step 3] 검증용 입력 객체(obj) 생성
        obj = {
            "slotName": config["slot"],
            "kind": "EXCEL",
            "ext": "xlsx",
            "period_start": PERIOD_START,
            "period_end": PERIOD_END,
            "dataframe": raw_df
        }
        
        # [Step 4] 메인 검증 파이프라인(L0~L4) 실행
        res = validate_structured_upstream(obj, file_path=config["file_name"])
        
        # [Step 5] Cell 4의 헬퍼를 통한 결과 출력
        print_full(res)
        
        # 결과 요약 리포트
        if res["status"] == "PASS":
            print(f"{label}: 모든 품질 게이트 통과 (PASS)")
        else:
            err = res.get("error", {})
            print(f"{label}: 검증 실패 (FAIL)")
            print(f"     - 사유: {err.get('code')} / {err.get('message')}")
            print(f"     - 위치: {err.get('location')}")

    except Exception as e:
        print(f"{label}: 런타임 오류 발생 -> {str(e)}")

print("\n" + "=" * 60)
print(f"[TEST FINISHED] 완료 시각: {now_utc_iso()}")

[REAL-FILE TEST] 검증 시작: 2026-01-21T02:44:55Z
------------------------------------------------------------

[PROCESSING] 전기요금 (성광벤드_전기요금_2025년_4분기.xlsx) 검사 중...
[MAIN] start
  - slotName: electricity_usage
  - file_path: 성광벤드_전기요금_2025년_4분기.xlsx
[L0] PASS
  - slotName: electricity_usage
  - shape: (8832, 6)
  - period_start: 2025-10-01 00:00:00
  - period_end: 2025-12-31 23:59:59
[L1] PASS
  - columns: ['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh', 'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor']
[L2] picked
  - date_col: date
  - flow_col: Usage_kWh
  - cum_col: None
  - unit_cols: ['Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh', 'Leading_Current_Reactive_Power_kVarh']
[L4] PASS
  - rows_in_period: 8832
{
  "status": "PASS",
  "file_path": "성광벤드_전기요금_2025년_4분기.xlsx",
  "payload": {
    "time_granularity": "15min",
    "unit_schema": [
      "time",
      "kWh",
      "-",
      "-",
      "-",
      "-"
    ],
 

Cell 14 — FAIL(L0) 테스트 2종
역할

입력 단계에서 FAIL이 제대로 발생하는지 증명

In [15]:
# ===== Cell 14: FAIL tests - L0 =====

print("=== [FAIL 기대] L0 - MISSING_KEYS ===")
obj = {"kind":"EXCEL","ext":"xlsx","period_start":PERIOD_START,"period_end":PERIOD_END,"dataframe":pd.DataFrame({"a":[1],"b":[2]})}
print_full(validate_structured_upstream(obj, file_path="L0_missing_keys.xlsx"))

print("\n=== [FAIL 기대] L0 - BAD_DF ===")
obj = {"slotName":"electricity_usage","kind":"EXCEL","ext":"xlsx","period_start":PERIOD_START,"period_end":PERIOD_END,"dataframe":"NOT_DF"}
print_full(validate_structured_upstream(obj, file_path="L0_bad_df.xlsx"))


=== [FAIL 기대] L0 - MISSING_KEYS ===
[MAIN] start
  - slotName: None
  - file_path: L0_missing_keys.xlsx
[L0] MISSING_KEYS
  - missing: ['slotName']
{
  "status": "FAIL",
  "error": {
    "code": "MISSING_KEYS",
    "message": "필수 키 누락: ['slotName']",
    "location": "input"
  },
  "file_path": "L0_missing_keys.xlsx",
  "processed_at": "2026-01-21T02:45:02Z"
}

=== [FAIL 기대] L0 - BAD_DF ===
[MAIN] start
  - slotName: electricity_usage
  - file_path: L0_bad_df.xlsx
[L0] BAD_DF_TYPE
  - got: <class 'str'>
{
  "status": "FAIL",
  "error": {
    "code": "BAD_DF",
    "message": "dataframe이 pandas.DataFrame이 아닙니다.",
    "location": "input.dataframe"
  },
  "file_path": "L0_bad_df.xlsx",
  "processed_at": "2026-01-21T02:45:02Z"
}


Cell 15 — FAIL(L1) 중복/빈 컬럼명
역할

L1 품질 게이트(중복/빈 컬럼명)가 실제로 FAIL을 만드는지 증명

In [16]:
# ===== Cell 15: FAIL tests - L1 =====

print("=== [FAIL 기대] L1 - 중복 컬럼명(strip 후 동일) ===")
df = pd.DataFrame({"A":[1]*10, " A ":[2]*10, "B":[3]*10})
obj = {"slotName":"electricity_usage","kind":"EXCEL","ext":"xlsx","period_start":PERIOD_START,"period_end":PERIOD_END,"dataframe":df}
print_full(validate_structured_upstream(obj, file_path="L1_dup_cols.xlsx"))

print("\n=== [FAIL 기대] L1 - 빈 컬럼명 ===")
df = pd.DataFrame({"":[1]*10, "B":[2]*10})
obj = {"slotName":"electricity_usage","kind":"EXCEL","ext":"xlsx","period_start":PERIOD_START,"period_end":PERIOD_END,"dataframe":df}
print_full(validate_structured_upstream(obj, file_path="L1_empty_col.xlsx"))


=== [FAIL 기대] L1 - 중복 컬럼명(strip 후 동일) ===
[MAIN] start
  - slotName: electricity_usage
  - file_path: L1_dup_cols.xlsx
[L0] PASS
  - slotName: electricity_usage
  - shape: (10, 3)
  - period_start: 2025-10-01 00:00:00
  - period_end: 2025-12-31 23:59:59
[L1] DUPLICATE_COLUMNS_AFTER_NORMALIZE
  - columns: ['A', 'A', 'B']
{
  "status": "FAIL",
  "error": {
    "code": "L1_INVALID_COLUMNS",
    "message": "L1 정규화 이후 중복 컬럼명이 발생했습니다(strip 후 동일).",
    "location": "df.columns"
  },
  "file_path": "L1_dup_cols.xlsx",
  "processed_at": "2026-01-21T02:45:02Z"
}

=== [FAIL 기대] L1 - 빈 컬럼명 ===
[MAIN] start
  - slotName: electricity_usage
  - file_path: L1_empty_col.xlsx
[L0] PASS
  - slotName: electricity_usage
  - shape: (10, 2)
  - period_start: 2025-10-01 00:00:00
  - period_end: 2025-12-31 23:59:59
[L1] EMPTY_COLUMN_NAME
  - columns: ['', 'B']
{
  "status": "FAIL",
  "error": {
    "code": "L1_INVALID_COLUMNS",
    "message": "빈 컬럼명이 존재합니다(strip 후 '').",
    "location": "df.columns"
  },
  "fi

Cell 16 — FAIL(L2) 필수 컬럼 탐지 실패
역할

L2에서 date/flow 후보를 못 찾으면 REQUIRED_FIELD_NOT_FOUND로 FAIL 나는지 증명

In [17]:
# ===== Cell 16: FAIL tests - L2 =====

print("=== [FAIL 기대] L2 - REQUIRED_FIELD_NOT_FOUND ===")
df = pd.DataFrame({
    "비정상_날짜필드": ["x"]*480,
    "알수없는_수치": ["foo"]*480,
    "메모": ["memo"]*480
})
obj = {"slotName":"electricity_usage","kind":"EXCEL","ext":"xlsx","period_start":"2026-02-01T00:00:00","period_end":"2026-02-05T23:45:00","dataframe":df}
print_full(validate_structured_upstream(obj, file_path="L2_required_not_found.xlsx"))


=== [FAIL 기대] L2 - REQUIRED_FIELD_NOT_FOUND ===
[MAIN] start
  - slotName: electricity_usage
  - file_path: L2_required_not_found.xlsx
[L0] PASS
  - slotName: electricity_usage
  - shape: (480, 3)
  - period_start: 2026-02-01 00:00:00
  - period_end: 2026-02-05 23:45:00
[L1] PASS
  - columns: ['비정상_날짜필드', '알수없는_수치', '메모']
[L2] AUTO_LLM_TRIGGERED
  - reason: rule_missing
  - USE_LLM: True
[L2] picked
  - date_col: 비정상_날짜필드
  - flow_col: 알수없는_수치
  - cum_col: None
  - unit_cols: []
{
  "status": "FAIL",
  "error": {
    "code": "TS_PARSE_LOW",
    "message": "날짜 파싱률 낮음(ts_rate=0.00)",
    "location": "col:비정상_날짜필드"
  },
  "file_path": "L2_required_not_found.xlsx",
  "processed_at": "2026-01-21T02:45:04Z"
}


Cell 17 — FAIL(L3) 단위 불일치(전기인데 m3)
역할

L3에서 단위 mismatch가 제대로 FAIL 처리되는지 증명

In [18]:
# ===== Cell 17: FAIL tests - L3 (UNIT_MISMATCH) =====

print("=== [FAIL 기대] L3 - UNIT_MISMATCH ===")
ts = pd.date_range("2026-01-10 00:00:00", "2026-01-23 23:45:00", freq="15min")
df = pd.DataFrame({
    "일시": ts,
    "사용량(m3)": np.round(np.random.uniform(10, 60, len(ts)), 3),  # 전기인데 m3로 들어온 케이스
    "비고": ["x"]*len(ts)
})
obj = {"slotName":"electricity_usage","kind":"EXCEL","ext":"xlsx","period_start":"2026-01-10T00:00:00","period_end":"2026-01-23T23:45:00","dataframe":df}
print_full(validate_structured_upstream(obj, file_path="L3_unit_mismatch.xlsx"))


=== [FAIL 기대] L3 - UNIT_MISMATCH ===
[MAIN] start
  - slotName: electricity_usage
  - file_path: L3_unit_mismatch.xlsx
[L0] PASS
  - slotName: electricity_usage
  - shape: (1344, 3)
  - period_start: 2026-01-10 00:00:00
  - period_end: 2026-01-23 23:45:00
[L1] PASS
  - columns: ['일시', '사용량(m3)', '비고']
[L2] picked
  - date_col: 일시
  - flow_col: 사용량(m3)
  - cum_col: None
  - unit_cols: []
{
  "status": "FAIL",
  "error": {
    "code": "UNIT_MISMATCH",
    "message": "단위 불일치: got=m3, expected=kWh",
    "location": "col:사용량(m3)"
  },
  "file_path": "L3_unit_mismatch.xlsx",
  "processed_at": "2026-01-21T02:45:04Z"
}


Cell 18 — FAIL(L4) 기간 내 timestamp 1개 누락
역할

L4에서 “기간 커버리지 100%”가 실제로 강제되는지 증명

In [19]:
# ===== Cell 18: FAIL tests - L4 (missing timestamp) =====

print("=== [FAIL 기대] L4 - PERIOD_MISSING_TIMESTAMPS ===")
ps = "2026-01-24T00:00:00"
pe = "2026-01-31T23:45:00"
ts = pd.date_range(pd.to_datetime(ps), pd.to_datetime(pe), freq="15min")
df = pd.DataFrame({
    "일시": ts,
    "사용량(kWh)": np.round(np.random.uniform(50, 180, len(ts)), 3),
    "역률": np.round(np.random.uniform(0.7, 1.0, len(ts)), 3),
})
df = df.drop(index=100).reset_index(drop=True)  # timestamp 1개 제거
obj = {"slotName":"electricity_usage","kind":"EXCEL","ext":"xlsx","period_start":ps,"period_end":pe,"dataframe":df}
print_full(validate_structured_upstream(obj, file_path="L4_missing_timestamp.xlsx"))


=== [FAIL 기대] L4 - PERIOD_MISSING_TIMESTAMPS ===
[MAIN] start
  - slotName: electricity_usage
  - file_path: L4_missing_timestamp.xlsx
[L0] PASS
  - slotName: electricity_usage
  - shape: (767, 3)
  - period_start: 2026-01-24 00:00:00
  - period_end: 2026-01-31 23:45:00
[L1] PASS
  - columns: ['일시', '사용량(kWh)', '역률']
[L2] picked
  - date_col: 일시
  - flow_col: 사용량(kWh)
  - cum_col: None
  - unit_cols: ['사용량(kWh)']
{
  "status": "FAIL",
  "error": {
    "code": "PERIOD_MISSING_TIMESTAMPS",
    "message": "기간 내 timestamp 누락",
    "location": "timestamp"
  },
  "file_path": "L4_missing_timestamp.xlsx",
  "processed_at": "2026-01-21T02:45:04Z"
}


Cell 20 - 테스트용 셀 (19보다 먼저 실행)

In [20]:
# ===== Cell 20: LLM Stub/Spy utilities =====

_LLM_ORIG_FUNCS = {}
_LLM_STUB_INSTALLED = False
_LLM_SPY_INSTALLED = False

_LLM_CALL_COUNTER = {"n": 0}

def llm_calls() -> int:
    return int(_LLM_CALL_COUNTER["n"])

def reset_llm_calls():
    _LLM_CALL_COUNTER["n"] = 0

def install_llm_stub(mapping: dict, llm_func_name: str = "llm_classify_columns_min_tokens"):
    """
    llm_classify_columns_min_tokens를 스텁으로 바꿔서
    항상 {"mapping": mapping}을 반환하게 함.
    - mapping 예: {"A":"date","B":"flow","C":"other"}
    """
    global _LLM_STUB_INSTALLED

    if llm_func_name not in globals():
        raise NameError(f"{llm_func_name} is not defined in globals(). 먼저 Cell 7(LLM 함수) 정의를 확인하세요.")

    if llm_func_name not in _LLM_ORIG_FUNCS:
        _LLM_ORIG_FUNCS[llm_func_name] = globals()[llm_func_name]

    def _stub(columns, df, slotName):
        _LLM_CALL_COUNTER["n"] += 1
        log("LLM_STUB", "CALLED", n=_LLM_CALL_COUNTER["n"], slotName=slotName, columns=len(columns))
        # 스텁은 무조건 매핑만 반환(설명 문장 X)
        return {"mapping": dict(mapping)}

    globals()[llm_func_name] = _stub
    _LLM_STUB_INSTALLED = True
    print("LLM stub installed.")

def install_llm_spy(llm_func_name: str = "llm_classify_columns_min_tokens"):
    """
    실제 llm_classify_columns_min_tokens 호출 횟수만 체크하는 spy.
    (원본을 감싸서 counter만 증가)
    """
    global _LLM_SPY_INSTALLED

    if llm_func_name not in globals():
        raise NameError(f"{llm_func_name} is not defined in globals().")

    if llm_func_name not in _LLM_ORIG_FUNCS:
        _LLM_ORIG_FUNCS[llm_func_name] = globals()[llm_func_name]

    orig = _LLM_ORIG_FUNCS[llm_func_name]

    def _spy(columns, df, slotName):
        _LLM_CALL_COUNTER["n"] += 1
        log("LLM_SPY", "CALLED", n=_LLM_CALL_COUNTER["n"], slotName=slotName, columns=len(columns))
        return orig(columns, df, slotName)

    globals()[llm_func_name] = _spy
    _LLM_SPY_INSTALLED = True
    print("LLM spy installed.")

def restore_llm(llm_func_name: str = "llm_classify_columns_min_tokens"):
    """
    stub/spy로 교체한 함수를 원복
    """
    global _LLM_STUB_INSTALLED, _LLM_SPY_INSTALLED

    if llm_func_name in _LLM_ORIG_FUNCS:
        globals()[llm_func_name] = _LLM_ORIG_FUNCS[llm_func_name]
        del _LLM_ORIG_FUNCS[llm_func_name]

    _LLM_STUB_INSTALLED = False
    _LLM_SPY_INSTALLED = False
    print("Restored.")


Cell 19 — LLM “호출 여부”를 확실히 증명하는 테스트(스파이/스텁 이용)
역할

팀원이 제일 많이 묻는 게 이거임:
“LLM이 진짜 호출되긴 해?”

이 셀은 force_llm=True로 LLM 경로를 강제로 태우고,
Cell 20의 spy/stub 카운트로 ‘진짜 호출됐다’를 증명한다.

In [21]:
# ===== Cell 19: LLM call proof (NO force_llm param) =====

reset_llm_calls()
USE_LLM = True

# 1) LLM이 항상 date/flow를 이렇게 찍도록 스텁 설치
install_llm_stub(mapping={"A":"date", "B":"flow", "C":"other"}, llm_func_name="llm_classify_columns_min_tokens")

# 2) 룰/폴백이 못 맞추게: 컬럼명 A,B,C + 데이터도 일부러 애매하게(신호 부족)
#    - A: 날짜처럼 보이지만 일부는 깨짐 -> dt_rate 낮게
#    - B: 숫자처럼 보이지만 일부는 문자 -> num_rate 낮게
ps = "2026-02-01T00:00:00"
pe = "2026-02-05T23:45:00"
ts = pd.date_range(pd.to_datetime(ps), pd.to_datetime(pe), freq="15min")

A = ts.astype(str).tolist()
# 날짜 일부 깨기
for i in np.random.choice(len(A), size=40, replace=False):
    A[i] = "날짜아님"

B = np.round(np.random.uniform(10, 60, len(ts)), 3).astype(object)
# 숫자 일부 깨기
for i in np.random.choice(len(B), size=40, replace=False):
    B[i] = "수치아님"

df = pd.DataFrame({"A": A, "B": B, "C": ["memo"]*len(ts)})

# 3) L2 호출: 현재 너의 L2는 (rule 부족 && USE_LLM=True)면 LLM 호출함
date_col, value_cols, unit_cols, err2, dbg = layer2_find_columns(df, slotName="electricity_usage")

print("used_llm =", dbg.get("used_llm"))
print("date_col =", date_col)
print("flow_col =", value_cols.get("flow"))
print("LLM_CALLS =", llm_calls())
print("err2 =", err2)

LLM stub installed.
[L2] AUTO_LLM_TRIGGERED
  - reason: rule_missing
  - USE_LLM: True
[LLM_STUB] CALLED
  - n: 1
  - slotName: electricity_usage
  - columns: 3
[L2] picked
  - date_col: A
  - flow_col: B
  - cum_col: None
  - unit_cols: []
used_llm = True
date_col = A
flow_col = B
LLM_CALLS = 1
err2 = None


CELL 21: LLM 테스트

In [22]:
# ===== Cell 21: LLM Reasoning Test (Inference Check) =====

import pandas as pd
import numpy as np

def run_llm_inference_test():
    # 1. 전역 설정 확인 (Cell 2의 설정이 True여야 함)
    if not USE_LLM:
        print("⚠️ 경고: USE_LLM이 False입니다. 테스트를 위해 True로 변경해주세요.")
        # return

    print(f"🚀 [LLM TEST] 시작 시각: {now_utc_iso()}")
    
    # 2. 룰 기반 엔진이 절대 맞출 수 없는 '암호화된' 데이터 생성
    # 컬럼명에 'date', 'usage' 같은 힌트가 전혀 없음
    data_len = 24  # 테스트용 24시간 데이터
    test_df = pd.DataFrame({
        "X_VAR_999": pd.date_range("2025-12-01", periods=data_len, freq="h"), # 실제: 날짜
        "Y_VAL_001": np.random.uniform(50, 100, size=data_len),               # 실제: 사용량(flow)
        "Z_REF_ETC": ["N/A"] * data_len                                      # 실제: 잡음 데이터
    })

    # 3. 검증 객체 구성
    obj_test = {
        "slotName": "electricity_usage",
        "kind": "EXCEL",
        "ext": "xlsx",
        "period_start": "2025-12-01T00:00:00",
        "period_end": "2025-12-01T23:00:00",
        "dataframe": test_df
    }

    print(f"📊 테스트 컬럼명: {list(test_df.columns)}")
    print("🤖 LLM에게 데이터 샘플을 보내 추론을 요청합니다...")

    # 4. 검증 엔진 실행 (L2에서 LLM이 개입해야 함)
    result = validate_structured_upstream(obj_test, file_path="inference_test.xlsx")

    # 5. 결과 출력 및 분석
    print_full(result)
    
    if result["status"] == "PASS":
        print("\n✨ 결과 분석: LLM이 암호화된 컬럼을 성공적으로 매핑했습니다.")
    else:
        print(f"\n🛑 결과 분석: {result['error']['code']} - {result['error']['message']}")

# 실행
run_llm_inference_test()

restore_llm()
USE_LLM = False

🚀 [LLM TEST] 시작 시각: 2026-01-21T02:45:04Z
📊 테스트 컬럼명: ['X_VAR_999', 'Y_VAL_001', 'Z_REF_ETC']
🤖 LLM에게 데이터 샘플을 보내 추론을 요청합니다...
[MAIN] start
  - slotName: electricity_usage
  - file_path: inference_test.xlsx
[L0] PASS
  - slotName: electricity_usage
  - shape: (24, 3)
  - period_start: 2025-12-01 00:00:00
  - period_end: 2025-12-01 23:00:00
[L1] PASS
  - columns: ['X_VAR_999', 'Y_VAL_001', 'Z_REF_ETC']
[L2] AUTO_LLM_TRIGGERED
  - reason: rule_missing
  - USE_LLM: True
[LLM_STUB] CALLED
  - n: 2
  - slotName: electricity_usage
  - columns: 3
[L2] picked
  - date_col: X_VAR_999
  - flow_col: Y_VAL_001
  - cum_col: None
  - unit_cols: []
[L4] PASS
  - rows_in_period: 24
{
  "status": "PASS",
  "file_path": "inference_test.xlsx",
  "payload": {
    "time_granularity": "hourly",
    "unit_schema": [
      "time",
      "kWh",
      "-"
    ],
    "validated_fields": [
      "timestamp",
      "flow_kwh",
      "Z_REF_ETC"
    ]
  },
  "processed_at": "2026-01-21T02:45:04Z"
}

✨ 결과 분석: LLM이 암호화된

C:\Users\User\AppData\Local\Temp\ipykernel_145500\2953140546.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  [(c, float(pd.to_datetime(df[c], errors="coerce").notna().mean())) for c in cols],
